In [1]:
%cd /content/drive/MyDrive/데청캠 5조/Test_1

/content/drive/MyDrive/데청캠 5조/Test_1


In [2]:
!pwd

/content/drive/MyDrive/데청캠 5조/Test_1


In [3]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import datetime
import time
from tqdm import notebook
import re
import glob

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3227648/45929032 bytes (7.0%)7225344/45929032 bytes (15.7%)11206656/45929032 bytes (24.4%)15220736/45929032 bytes (33.1%)19177472/45929032 bytes (41.8%)22921216/45929032 bytes (49.9%)26533888/45929032 bytes (57.8%)30457856/45929032 bytes (66.3%)34439168/45929032 bytes (75.0%)38346752/45929032 bytes (83.5%)42098688/45929032 bytes (91.7%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [4]:
# MOV_List : 영상 데이터 저장하는 변수
MOV_List = np.array(glob.glob('./MOV/*.mp4'))
print(MOV_List)
print('MOV 파일 개수 :', len(MOV_List))

['./MOV/readnaversid1370oid214aid0000577019.mp4'
 './MOV/readnaversid1370oid214aid0000577018.mp4'
 './MOV/readnaversid1370oid214aid0000577275.mp4'
 './MOV/readnaversid1370oid214aid0000577284.mp4'
 './MOV/readnaversid1370oid214aid0000577286.mp4'
 './MOV/readnaversid1742oid437aid0000011278.mp4'
 './MOV/readnaversid1370oid214aid0000577293.mp4'
 './MOV/readnaversid1742oid437aid0000011415.mp4'
 './MOV/readnaversid1371oid055aid0000298141.mp4'
 './MOV/readnaversid1371oid055aid0000298142.mp4'
 './MOV/readnaversid1371oid055aid0000298152.mp4'
 './MOV/readnaversid1371oid055aid0000298230.mp4'
 './MOV/readnaversid1371oid055aid0000298236.mp4'
 './MOV/readnaversid1750oid448aid0000195576.mp4' './MOV/video.mp4'
 './MOV/readnaversid1750oid448aid0000196603.mp4'
 './MOV/readnaversid1750oid448aid0000310334.mp4'
 './MOV/readnaversid1750oid448aid0000195268.mp4'
 './MOV/readnaversid1742oid437aid0000011423.mp4']
MOV 파일 개수 : 19


In [5]:
# Two_min_under_MOV 함수 : 2분 이하 동영상 선별 

# 2분 이하 영상만 선별하는 이유 : 2분 넘는 영상들은 스페셜 영상이 주를 이루고 있음. 
# (앵커-리포터 형식의 영상이 아님) 

def Two_min_under_MOV(MOV_List):
    
    # Using_List : 사용할 영상들 file_name 모음 
    Using_List = MOV_List[:]
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        clip = VideoFileClip(MOV_List[i])
        play_time = clip.duration
        
        # 재생 시간이 2분을 넘는 경우 -> Using_list에서 제외
        if play_time > 60*2 :
            print('Except MOV_file name :', Using_List[i])
            del_index = np.append(del_index, [i])
            
    Using_List = np.delete(Using_List, del_index)
            
    # 2분 미만인 동영상 리스트
    return Using_List

In [6]:
Using_MOV_List = Two_min_under_MOV(MOV_List)
print(Using_MOV_List)
print('2분 이하 파일 개수', len(Using_MOV_List))

Except MOV_file name : ./MOV/readnaversid1370oid214aid0000577284.mp4
Except MOV_file name : ./MOV/readnaversid1370oid214aid0000577286.mp4
Except MOV_file name : ./MOV/readnaversid1750oid448aid0000195268.mp4

['./MOV/readnaversid1370oid214aid0000577019.mp4'
 './MOV/readnaversid1370oid214aid0000577018.mp4'
 './MOV/readnaversid1370oid214aid0000577275.mp4'
 './MOV/readnaversid1742oid437aid0000011278.mp4'
 './MOV/readnaversid1370oid214aid0000577293.mp4'
 './MOV/readnaversid1742oid437aid0000011415.mp4'
 './MOV/readnaversid1371oid055aid0000298141.mp4'
 './MOV/readnaversid1371oid055aid0000298142.mp4'
 './MOV/readnaversid1371oid055aid0000298152.mp4'
 './MOV/readnaversid1371oid055aid0000298230.mp4'
 './MOV/readnaversid1371oid055aid0000298236.mp4'
 './MOV/readnaversid1750oid448aid0000195576.mp4' './MOV/video.mp4'
 './MOV/readnaversid1750oid448aid0000196603.mp4'
 './MOV/readnaversid1750oid448aid0000310334.mp4'
 './MOV/readnaversid1742oid437aid0000011423.mp4']
2분 이하 파일 개수 16


In [7]:
# one_anchor_starts_MOV 함수 : 앵커 1명의 얼굴로 시작되는 영상 리스트 찾기
def one_anchor_starts_MOV(MOV_List):
     
    Using_MOV = MOV_List[:]
    
    # 가중치 파일 경로
    cascade_filename = 'haarcascade_frontalface_alt.xml'
    # 모델 불러오기
    cascade = cv2.CascadeClassifier(cascade_filename)
    
    # 첫 장면에서 인식한 얼굴의 수
    face_num_list = np.array([])
    face_num_list = face_num_list.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        video = cv2.VideoCapture(MOV_List[i]) #video name change
        ret, frame = video.read()
        
        # 첫 번째로 등장하는 프레임만 취급
        frame[0]

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        results = cascade.detectMultiScale(gray,
                                          scaleFactor=1.3,
                                          minNeighbors=5, minSize=(20,20))
        
        face_num_list = np.append(face_num_list, [len(results)])
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    # 앵커 1명으로 시작하는 영상 찾기
    for k in range(len(face_num_list)):
        if face_num_list[k]!=1:
            del_index = np.append(del_index, [k])
    
    Using_MOV = np.delete(Using_MOV, del_index)
    
    # 파일 처리 쉽게 할 수 있는 것을 목표 
    # -> MOV 원본 파일 주소, MOV 파일확장자 지운 버전 모두 제공 
    return Using_MOV

In [8]:
Using_MOV_List = one_anchor_starts_MOV(Using_MOV_List)
print(Using_MOV_List)
print('앵커 1명 얼굴로 시작되는 2분 이하의 영상', len(Using_MOV_List))


['./MOV/readnaversid1370oid214aid0000577019.mp4'
 './MOV/readnaversid1370oid214aid0000577018.mp4'
 './MOV/readnaversid1742oid437aid0000011278.mp4'
 './MOV/readnaversid1370oid214aid0000577293.mp4'
 './MOV/readnaversid1742oid437aid0000011415.mp4'
 './MOV/readnaversid1371oid055aid0000298141.mp4'
 './MOV/readnaversid1371oid055aid0000298142.mp4'
 './MOV/readnaversid1371oid055aid0000298152.mp4'
 './MOV/readnaversid1750oid448aid0000195576.mp4'
 './MOV/readnaversid1750oid448aid0000310334.mp4'
 './MOV/readnaversid1742oid437aid0000011423.mp4']
앵커 1명 얼굴로 시작되는 2분 이하의 영상 11


In [9]:
# Clean_File_List 변수 : 소속 위치, 확장자 뗀 본연의 파일명 저장
Clean_File_List = np.array([])

for File in Using_MOV_List:
  File_name = re.sub(pattern='./MOV/', repl='', string=File)
  File_name = re.sub(pattern='.mp4', repl='', string=File_name)
  Clean_File_List = np.append(Clean_File_List, [File_name])

print(Clean_File_List)
print('소속 위치, 확장자 뗀 본연의 파일명 개수 : ', len(Clean_File_List))

['readnaversid1370oid214aid0000577019'
 'readnaversid1370oid214aid0000577018'
 'readnaversid1742oid437aid0000011278'
 'readnaversid1370oid214aid0000577293'
 'readnaversid1742oid437aid0000011415'
 'readnaversid1371oid055aid0000298141'
 'readnaversid1371oid055aid0000298142'
 'readnaversid1371oid055aid0000298152'
 'readnaversid1750oid448aid0000195576'
 'readnaversid1750oid448aid0000310334'
 'readnaversid1742oid437aid0000011423']
소속 위치, 확장자 뗀 본연의 파일명 개수 :  11


In [ ]:
!pip install git+https://github.com/pyannote/pyannote-audio

In [11]:
# MOV_to_WAV 함수 : 영상(.mp4) -> 음성(.wav) 전환 작업
def MOV_to_WAV(MOV_List):
    WAV_file = np.array([])
    
    for MOV in notebook.tqdm(MOV_List): 
        clip = VideoFileClip(MOV)

        # 파일 저장 위치 변경
        sound_file_name = re.sub(pattern='MOV', repl='WAV', string=str(MOV))
        # 파일 저장 형태 변경
        sound_file_name = re.sub(pattern='mp4', repl='wav', string=sound_file_name)
        
        clip.audio.write_audiofile(sound_file_name)

In [12]:
MOV_to_WAV(Using_MOV_List)
print('wav 변환 성공 파일 수 :', len(glob.glob('./WAV/*.wav')))

[MoviePy] Writing audio in ./WAV/readnaversid1370oid214aid0000577019.wav


100%|██████████| 2623/2623 [00:01<00:00, 2155.50it/s]

[MoviePy] Done.


[MoviePy] Writing audio in ./WAV/readnaversid1370oid214aid0000577018.wav


100%|██████████| 2147/2147 [00:00<00:00, 2164.06it/s]

...

100%|██████████| 2057/2057 [00:01<00:00, 1873.74it/s]

[MoviePy] Done.


[MoviePy] Writing audio in ./WAV/readnaversid1742oid437aid0000011423.wav


100%|██████████| 2044/2044 [00:01<00:00, 1774.95it/s]

[MoviePy] Done.

wav 변환 성공 파일 수 : 11


In [13]:
import torch
import pyannote
from pyannote.database import get_protocol
from pyannote.database import FileFinder
from pyannote.audio.utils.signal import Binarize

from datetime import datetime, timedelta
import time

In [14]:
# sad 모델 설정
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')
preprocessors = {'audio':FileFinder()}
protocol = get_protocol('AMI.SpeakerDiarization.MixHeadset',
                            preprocessors=preprocessors)
    
binarize = Binarize(offset=0.52, onset=0.52, log_scale=True,
                    min_duration_off=0.1, min_duration_on=0.1)

Downloading: "https://github.com/pyannote/pyannote-audio/archive/master.zip" to /root/.cache/torch/hub/master.zip


In [15]:
# Usig_sad 함수 : sad 모델 사용 함수
def Using_sad(WAV_File):
  test_file = {}
  test_file['audio'] = WAV_File
  sad_scores = sad(test_file)

  speech = binarize.apply(sad_scores, dimension=1)
  
  # TimeStamp 편집 과정 (기본값 : 0번째 소리를 선택)
  a = str(speech[0])
  a = re.sub(pattern='\-\-\>', repl='', string=a)
  a = re.sub(pattern='\[', repl='', string=a)
  a = re.sub(pattern='\]', repl='', string=a)
  a = str.split(a)
  
  b = a[1]
  b = b.split(sep=':')
  
  target_sec = 0
  index = 0
  for i in reversed(b):
    target_sec += float(i) * (60**index)
    index += 1

  # 0번째 소리가 잡음일 경우에는 1번째 소리를 선택
  # target_sec < 1.5인 이유 : 잡음은 보통 1.5초 전에 인식이 끝나기 때문
  # 주절주절 tmi : 음성 편집 기술이 덜 발달한 2010년대 초반 앵커 영상에 주로 그런 잡음이 들어 있음.
  if target_sec < 1.5:
    a = str(speech[1])
    a = re.sub(pattern='\-\-\>', repl='', string=a)
    a = re.sub(pattern='\[', repl='', string=a)
    a = re.sub(pattern='\]', repl='', string=a)
    a = str.split(a)

    b = a[1]
    b = b.split(sep=':')

    target_sec = 0
    index = 0
    for i in reversed(b):
      target_sec += float(i) * (60**index)
      index += 1
  
  # 앵커-리포터 전환 지점의 60%에 해당하는 곳은 앵커의 입모양, 말소리를 확실하게 캐치할 수 있음.
  target_sec = target_sec * 0.6
  print(WAV_File, ':', target_sec, 'sec')

  return WAV_File, target_sec        
  

In [16]:
!pip install pydub

In [17]:
!pip install speechrecognition

     |████████████████████████████████| 32.8 MB 35 kB/s 


In [18]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import re
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import speech_recognition as sr

In [19]:
# 8/11 수정 완료! 얼굴 인식 완성도만 조금 더 높이면 될 것 같음! 
# (대체로 잘 인식하긴 하지만, 가끔씩 뒤에 있는 인물 사진을 앵커라고 인식하는 경우가 있음)
def face_crop_vid (Clean_File_name):
  # 얼굴 인식 모델 가중치 파일
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

  # [ 1단계 ] 얼굴 좌표 구하기 - 첫 프레임만 사용
  # 얼굴 좌표 구할 파일 정보
  File_name = './CUT_VID/' + Clean_File_name + '.mp4'
  print(File_name)
  video1 =  cv2.VideoCapture(File_name)

  # 얼굴 좌표 구하기 작업
  ret, pic = video1.read()
  pic[0]

  gray = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 
                                        scaleFactor=1.3, 
                                        minNeighbors=5, minSize=(20,20))
  
  face = faces[0]
  x, y, w, h = face
  
  video1.release()


  # [2단계] 비디오 저장 처리 - 영상 속 모든 프레임 사용 
  video2 =  cv2.VideoCapture(File_name)
  
  # 저장할 파일 양식 지정
  fps = video2.get(cv2.CAP_PROP_FPS)
  fourcc = cv2.VideoWriter_fourcc(*'DIVX')
  width = video2.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = video2.get(cv2.CAP_PROP_FRAME_HEIGHT)
  size = (round(width), round(height))
  out = cv2.VideoWriter('./FINAL_VID/%s.avi' %Clean_File_name, fourcc, fps, size)
    
  # 파일 처리 작업
  while (video2.isOpened()):
    ret, frame = video2.read()

    if not ret:
        break

    cropped = frame[y - int(h/4):y + h + int(h/4), 
                    x - int(w/4):x + w + int(w/4)]
    
    cropped = cv2.resize(cropped, 
                         dsize=(round(width), round(height)), 
                         interpolation=cv2.INTER_LINEAR)

    # 위에서 자른 얼굴 위치를 토대로 프레임 저장 (연속적 프레임이므로 영상 형태)
    out.write(cropped)

  out.release()
  video2.release()


In [20]:
## 파일 전체 대상으로 코드 돌리기 
# (문장 단위로 끊기에는 문장과 문장을 연달아 하는 발음이 많아서 앵커 대사 기준으로 끊어봤음)

for File in notebook.tqdm(Clean_File_List):
  # 시간 좌표 구하기 (통문장)
  WAV_File = './WAV/' + File + '.wav'
  print(WAV_File)
  WAV_File, target_sec = Using_sad(WAV_File)

  # target_sec * 0.6 과정을 거쳤는데도 값이 20을 넘기면 그건 리포터가 쭉 말하고 있는 거임.
  # 따라서 이런 영상(앵커 없이 리포터만 말하는 것)들은 제외해야 함. 
  if target_sec > 20:
    print('앵커 없이 리포터만 말하는 영상이므로 skip')
    continue

  MOV_File = './MOV/' + File + '.mp4'
  print(MOV_File)

  # 시간 좌표 토대로 영상 자르기 
  clip = mp.VideoFileClip(MOV_File)
  video_clip = clip.subclip(0, target_sec)
  video_clip.write_videofile('./CUT_VID/' + File + '.mp4')

  clip2 = mp.VideoFileClip('./CUT_VID/' + File + '.mp4')
  clip2.audio.write_audiofile('./CUT_WAV/' + File + '.wav')

  # 대본 파일 (대본 정확도를 어떤 식으로 높일지 고민하는 중)
  # 지금 정확도가 낮다는 소리는 아니지만, 지도학습 정확도를 높이기 위해서는 대본이 더 정확해야 할 것 같음!
  # 원본 텍스트 파일을 가져와서 앵커 부분만 빼낼까 생각하는 중...
  # 텍스트 변환 정확도 분석할 때 원본에서 띄어쓰기/문장부호 다 빼고 len(변환 텍스트)까지의 정확도 측정하라고 하면 될 것 같음.
  audio = sr.AudioFile('./CUT_WAV/' + File + '.wav')

  recognizer = sr.Recognizer()
  recognizer.energy_threshold = 300
  
  with audio as source:
    audio_news = recognizer.record(source)

  text = recognizer.recognize_google(audio_data=audio_news, language="ko-KR", show_all=False)

  f = open('./CUT_TXT/' + File + '.txt', 'w')
  text = str(text)
  f.write(text)
  f.close()

  face_crop_vid(File)

./WAV/readnaversid1370oid214aid0000577019.wav
./WAV/readnaversid1370oid214aid0000577019.wav : 9.4908 sec
./MOV/readnaversid1370oid214aid0000577019.mp4
[MoviePy] >>>> Building video ./CUT_VID/readnaversid1370oid214aid0000577019.mp4
[MoviePy] Writing audio in readnaversid1370oid214aid0000577019TEMP_MPY_wvf_snd.mp3


100%|██████████| 210/210 [00:00<00:00, 795.24it/s]

[MoviePy] Done.
[MoviePy] Writing video ./CUT_VID/readnaversid1370oid214aid0000577019.mp4



100%|██████████| 285/285 [00:04<00:00, 63.73it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./CUT_VID/readnaversid1370oid214aid0000577019.mp4 

[MoviePy] Writing audio in ./CUT_WAV/readnaversid1370oid214aid0000577019.wav


...


100%|██████████| 145/145 [00:01<00:00, 132.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./CUT_VID/readnaversid1742oid437aid0000011423.mp4 

[MoviePy] Writing audio in ./CUT_WAV/readnaversid1742oid437aid0000011423.wav


100%|██████████| 134/134 [00:00<00:00, 1928.61it/s]

[MoviePy] Done.


./CUT_VID/readnaversid1742oid437aid0000011423.mp4

